1. Preparación de los datos

In [1]:
import pandas as pd

data = pd.read_csv('data/normalized_merged_data.csv')

2. Limpieza y preprocesamiento de texto

In [2]:
#!pip install spacy

In [3]:
#!python -m spacy download en_core_web_sm

In [4]:
# import spacy

# # Carga el modelo en inglés de spaCy
# nlp = spacy.load('en_core_web_sm')

In [5]:
# def preprocess_text(text):
#     # Aplica el modelo de spaCy al texto
#     doc = nlp(text)

#     # Tokenización y lematización
#     preprocessed_text = [token.lemma_ for token in doc if not token.is_stop]

#     # Combina las palabras preprocesadas en un solo texto
#     preprocessed_text = ' '.join(preprocessed_text)

#     return preprocessed_text

In [6]:
# data['text'] = data['text'].apply(preprocess_text)

3. Carga del modelo GPT-2

In [7]:
from transformers import GPT2ForSequenceClassification, GPT2Tokenizer

# Carga el modelo pre-entrenado y el tokenizador
model = GPT2ForSequenceClassification.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

C:\Users\Jose\AppData\Roaming\Python\Python39\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
tokenizer.pad_token = tokenizer.eos_token

# Tokeniza tus resúmenes
train_encodings = tokenizer(train_text, truncation=True, padding=True)
test_encodings = tokenizer(test_text, truncation=True, padding=True)

# Agregar un token de padding personalizado, por ejemplo, '[PAD]'
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

train_encodings = tokenizer(train_text, truncation=True, padding=True)
test_encodings = tokenizer(test_text, truncation=True, padding=True)


In [23]:
# Define una función para ajustar las secuencias al límite de longitud
def adjust_sequence_length(text, max_length=1024):
    if len(text) > max_length:
        # Trunca o divide el texto si es demasiado largo
        return text[:max_length]
    else:
        return text

# Aplica la función de ajuste a tus datos
data['text'] = data['text'].apply(adjust_sequence_length)


4. Entrenamiento

In [26]:
import pandas as pd
import torch
from transformers import GPT2ForSequenceClassification, GPT2Tokenizer
from sklearn.model_selection import train_test_split

# Carga tus datos desde el archivo CSV
data = pd.read_csv('data/normalized_merged_data.csv')

# Agrega una nueva columna de etiquetas con los valores 0 por defecto
data['label'] = 0

# Aplica las condiciones para asignar etiquetas 1
data.loc[(data['normalized_content'] > 0.7) & (data['normalized_wording'] > 0.8), 'label'] = 1


# Define las columnas de entrada (text) y las etiquetas
text = data['text']
labels = data['label']  # Supongamos que tienes una columna 'label' que indica si es correcto o no

# Divide los datos en conjuntos de entrenamiento y prueba
train_text, test_text, train_labels, test_labels = train_test_split(text, labels, test_size=0.2, random_state=42)

# Carga el modelo GPT-2 y el tokenizador
model = GPT2ForSequenceClassification.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Agrega un token de padding personalizado, por ejemplo, '[PAD]'
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Tokeniza tus datos y ajusta la longitud
train_encodings = tokenizer(list(train_text), truncation=True, padding=True, max_length=512, return_tensors='pt')
test_encodings = tokenizer(list(test_text), truncation=True, padding=True, max_length=512, return_tensors='pt')

# Convierte las etiquetas a tensores
train_labels = torch.tensor(list(train_labels))
test_labels = torch.tensor(list(test_labels))

# Añade una capa de clasificación lineal en la parte superior del modelo GPT-2
model.classifier = torch.nn.Linear(in_features=model.config.hidden_size, out_features=2)  # 2 clases: correcto o incorrecto

# Entrena el modelo
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
train_dataset = torch.utils.data.TensorDataset(train_encodings.input_ids, train_encodings.attention_mask, train_labels)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True)

num_epochs = 3  # Ajusta el número de épocas según tus necesidades

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        input_ids, attention_mask, labels = batch
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Average Loss: {total_loss / len(train_dataloader)}")

# Evalúa el modelo en el conjunto de prueba
model.eval()
with torch.no_grad():
    test_dataset = torch.utils.data.TensorDataset(test_encodings.input_ids, test_encodings.attention_mask, test_labels)
    test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=8, shuffle=False)
    correct = 0
    total = 0
    for batch in test_dataloader:
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids, attention_mask=attention_mask)
        predicted_labels = torch.argmax(outputs.logits, dim=1)
        correct += (predicted_labels == labels).sum().item()
        total += labels.size(0)

accuracy = correct / total
print(f'Accuracy: {accuracy}')

# Guarda el modelo entrenado si estás satisfecho con el rendimiento
model.save_pretrained("trained_gpt2_model")


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


IndexError: index out of range in self

In [24]:
import torch
from sklearn.model_selection import train_test_split

umbral_content = 0.7  # Ejemplo, puedes ajustar este valor
umbral_wording = 0.8  # Ejemplo, puedes ajustar este valor

# Define las etiquetas basadas en umbrales
data['label'] = (data['content'] >= umbral_content) & (data['wording'] >= umbral_wording)
data['label'] = data['label'].astype(int)  # Convierte a 0 o 1

train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Define las etiquetas de entrenamiento y prueba
train_labels = train_data['label'].tolist()
test_labels = test_data['label'].tolist()

# Convierte las listas en tensores de PyTorch
train_input_ids = torch.tensor(train_encodings.input_ids)
train_attention_mask = torch.tensor(train_encodings.attention_mask)
train_labels = torch.tensor(train_labels)

# Añade una capa de clasificación lineal en la parte superior del modelo GPT-2
model.classifier = torch.nn.Linear(in_features=model.config.hidden_size, out_features=2)

# Entrena el modelo
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
# Crea el conjunto de datos y el dataloader
train_dataset = torch.utils.data.TensorDataset(train_input_ids, train_attention_mask, train_labels)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)

# Define el número de épocas
epochs = 5  # Puedes ajustar este valor según tus necesidades

for epoch in range(epochs):
    for batch in train_dataloader:
        input_ids, attention_mask, labels = batch
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()


IndexError: index out of range in self

5. Evaluación del modelo

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# Evalúa el modelo en el conjunto de prueba
with torch.no_grad():
    outputs = model(test_encodings.input_ids, attention_mask=test_encodings.attention_mask)
    predicted_labels = torch.argmax(outputs.logits, dim=1)
    accuracy = accuracy_score(test_labels, predicted_labels)
    report = classification_report(test_labels, predicted_labels)
    
print(f'Accuracy: {accuracy}')
print(report)
